# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_Keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,18.44,82,0,3.09,US,1707835703
1,1,anan,33.9167,134.6500,6.88,82,0,1.70,JP,1707835703
2,2,nar'yan-mar,67.6713,53.0870,-29.38,98,71,3.58,RU,1707835704
3,3,malango,-2.9627,119.9001,20.78,98,100,0.66,ID,1707835704
4,4,karratha,-20.7377,116.8463,27.13,79,5,4.26,AU,1707835705


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Configure the map plot
city_map = city_data_df.hvplot.points('Lng',
                                    'Lat',
                                    geo = True,
                                    tiles = 'OSM',
                                    frame_width = 800,
                                    frame_height = 600,
                                    size = 'Humidity',
                                    color = 'City',
                                    hover_cols = ['City','Humidity'])

# # Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
city_data_df.count()

City_ID       579
City          579
Lat           579
Lng           579
Max Temp      579
Humidity      579
Cloudiness    579
Wind Speed    579
Country       576
Date          579
dtype: int64

In [15]:
city_data_df = city_data_df.dropna()
city_data_df.count()

City_ID       576
City          576
Lat           576
Lng           576
Max Temp      576
Humidity      576
Cloudiness    576
Wind Speed    576
Country       576
Date          576
dtype: int64

In [16]:
# Create a variable to store preferred cities
pref_city_df = city_data_df[(city_data_df['Max Temp'] < 25) & (city_data_df['Max Temp'] > 18) & (city_data_df['Cloudiness'] == 0)]

pref_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,18.44,82,0,3.09,US,1707835703
5,5,kapa'a,22.0752,-159.3190,19.18,82,0,3.09,US,1707835705
44,44,kone,-21.0595,164.8658,24.27,90,0,2.07,NC,1707835725
50,50,brevard,28.2920,-80.7492,20.03,78,0,7.20,US,1707835706
68,68,lihue,21.9789,-159.3672,18.77,88,0,3.09,US,1707835737
126,126,port macquarie,-31.4333,152.9167,19.38,75,0,5.92,AU,1707835515
137,137,port elizabeth,-33.9180,25.5701,23.16,78,0,11.32,ZA,1707835772
178,178,general acha,-37.3770,-64.6043,21.62,26,0,4.61,AR,1707835793
186,186,geraldton,-28.7667,114.6000,22.23,83,0,5.66,AU,1707835797
235,235,general conesa,-40.1000,-64.4333,24.92,16,0,6.25,AR,1707835824


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pref_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wailua homesteads,US,22.0669,-159.3780,82,
5,kapa'a,US,22.0752,-159.3190,82,
44,kone,NC,-21.0595,164.8658,90,
50,brevard,US,28.2920,-80.7492,78,
68,lihue,US,21.9789,-159.3672,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1

params = {
    'categories':categories,
    'api_key':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = pref_city_df.loc[index, 'Lat']
    longitude = pref_city_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kapa'a - nearest hotel: Pono Kai Resort
kone - nearest hotel: Pacifik Appartels
brevard - nearest hotel: Home2 Suites - Melbourne Viera
lihue - nearest hotel: Kauai Palms
port macquarie - nearest hotel: The Observatory
port elizabeth - nearest hotel: Waterford Hotel
general acha - nearest hotel: Hotel Patagonia
geraldton - nearest hotel: Ocean Centre Hotel
general conesa - nearest hotel: Gran Hotel Conesa
north tamworth - nearest hotel: Imperial Brew House
toormina - nearest hotel: Boambee Bay Resort
dalbandin - nearest hotel: SF Al-dawood Restaurant
al hoceima - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
tazacorte - nearest hotel: App Leyma
coyhaique - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
0,wailua homesteads,US,22.0669,-159.3780,82,Hilton Garden Inn Kauai Wailua Bay
5,kapa'a,US,22.0752,-159.3190,82,Pono Kai Resort
44,kone,NC,-21.0595,164.8658,90,Pacifik Appartels
50,brevard,US,28.2920,-80.7492,78,Home2 Suites - Melbourne Viera
68,lihue,US,21.9789,-159.3672,88,Kauai Palms
126,port macquarie,AU,-31.4333,152.9167,75,The Observatory
137,port elizabeth,ZA,-33.9180,25.5701,78,Waterford Hotel
178,general acha,AR,-37.3770,-64.6043,26,Hotel Patagonia
186,geraldton,AU,-28.7667,114.6000,83,Ocean Centre Hotel
235,general conesa,AR,-40.1000,-64.4333,16,Gran Hotel Conesa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng',
                                    'Lat',
                                    geo = True,
                                    tiles = 'OSM',
                                    frame_width = 800,
                                    frame_height = 600,
                                    color = 'City',
                                    hover_cols = ['Hotel Name','Country'])

# # Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)